In [4]:
import numpy as np
import pandas as pd
import random as rd

In [5]:
df_data = pd.read_csv('..\Voorbeeld_clusterdata.txt', sep='  ', header=None, engine='python', index_col=0)
df_results = pd.read_csv('..\Voorbeeld_clusterresult.txt', sep='  ', header=None, engine='python', index_col=0)
df_clusterd = df_data.copy().drop(columns=df_data.columns)
gen_ID = list(df_data.index.values)

In [6]:
def lib(FileName):
    def openFile(FileName):
        read_data = open(f'{FileName}')
        data = read_data.read()
        data = data.splitlines()
        read_data.close()
        
        return data
        
    lib = {}
    
    for lines in openFile(FileName):
        line = lines.split()
        lib[line[0]] = line[1:]
            
    return lib

lib_data = lib('..\Voorbeeld_clusterdata.txt')
lib_results = lib('..\Voorbeeld_clusterresult.txt')

In [7]:
class KMCA:
    def __init__(self, k=6, seed=20, max_iter=300):
        '''
        preconditions:  The number of clusters for, the number of seed for seed, the maximum number of times
                        you want to itterate over the data
        postconditions: sets the number of clusters to self.k
                        sets the number of seeds to self.seed
                        sets the number of maximum itterations to self.max_iter
        '''
        
        self.k = k + 1
        self.seed = seed
        self.max_iter = max_iter
        
#    def set_k(self, k):
#        self.k = k
        
#    def set_seed(self, seed):
#        self.seed = seed
        
    
    
    def fit(self, data):
        '''
        preconditions:  a library with that contains the data. the library must have tehe gene code as key and a list
                        of the datapoints as value.
        postconditions: returns a library: 
                            keys   --> clusters
                            values --> genes that are classefied in that cluster
        this functions task: to assign all genes to its right cluster.
        '''
        
        def E_score():
            '''
            preconditions:  /
            postconditions: returns the e_score as an integer.
            this functions task: calculate the e_score for the clusterd genes 
            '''
            mean = 0

            for k in range(1,self.k):
                score = np.sum([np.square(abs(np.subtract(self.centroids[k], self.lib_norm[dis]))) for dis in self.labeled_data[k]])
                mean += score

            e_score = mean / self.k
            return e_score

        def normalize():
            '''
            preconditions:  /
            postconditions: sets the normalised coordinates as content for the variable self.lib_norm
            this functions task: normalise the coordinates of all genes
            '''
            normalized = dict()
            for i in data:
                norm = np.sqrt(np.sum(np.square(np.array(data[i]).astype(float))))
                normalized[i] = np.divide(np.array(data[i]).astype(float), norm)

            self.lib_norm = normalized
        
        
        def rand_label():
            '''
            preconditions:  /
            postconditions: returs a library:
                                keys   --> cluster
                                values --> genes that are random classefied to that cluster
            this functions task: random assigning genes to a cluster'''
            
            labeld_data = dict()
            labels_id = dict()
            np.random.seed(self.seed)
            labels = np.random.randint(1,self.k,len(self.lib_norm))

            for i in range(len(self.lib_norm)):
                labeld_data[list(self.lib_norm.keys())[i]] = labels[i]

            for K in range(1,self.k):
                labels_id[K] = np.array([i for i,j in labeld_data.items() if j == K])
            return labels_id
            
        def centroid():
            '''
            preconditions:  /
            postconditions: returns a library:
                                keys   --> cluster
                                values --> list of coordinats for the center of that cluster
            this functions task: calculating the centers of the clusters'''
            centroids = dict()

            for k in range(1,self.k):
                centroids[k] = np.mean(np.array([self.lib_norm[index] for index in self.labeled_data[k]]), axis=0)
            return centroids
        
        def assign_cluster(unlabeld_data):
            '''
            preconditions:  a library:
                                keys  --> gen
                                value --> the normalised coordinates of the gen
            postconditions: a library:
                                keys  --> cluster
                                value --> genes classified to that cluster                     
            this functions task: assigning genes to the cluster with the nearest center'''
            lib_clust = {k:[] for k in range(1,self.k)}
            lijst = list()
            
            for cords in unlabeld_data:
                clust = np.argmin([np.sqrt(np.sum(np.square(np.subtract(self.centroids[k],unlabeld_data[cords])))) for k in range(1,self.k)])+1
                lib_clust[clust].append(cords)
            
            return lib_clust
        
        for iteration in range(self.max_iter):
            if iteration == 0:
                norm = normalize()
                self.labeled_data = rand_label()
                self.centroids = centroid()
                self.E_score = E_score()
                
            else:
                optimized = True
                
                prev_centroids = self.centroids
                self.labeled_data = assign_cluster(self.lib_norm)
                
                self.centroids = centroid()
                new_e_score = E_score()
            
                if self.E_score > new_e_score:
                    optimized = False
                    self.E_score = new_e_score
                        
                if optimized:
                    break
                    
        return self.labeled_data
    
    
    def predict(self, input_data):
        pass

In [11]:
set_variables = KMCA(seed=22)
result = set_variables.fit(lib_data)
for cluster in result:
    print(f'cluster {cluster}: bevat {len(result[cluster])} data punten')

cluster 1: bevat 239 data punten
cluster 2: bevat 43 data punten
cluster 3: bevat 106 data punten
cluster 4: bevat 90 data punten
cluster 5: bevat 412 data punten
cluster 6: bevat 122 data punten
